In [3]:
include("../output.jl")
using ImpvolOutput, FileIO
results = load("../experiments/baseline/actual/results.jld2")["results"]
parameters = load("../experiments/baseline/common_parameters.jld2")["parameters"]
real_GDP_model = sum(ImpvolOutput.make_series(results, :real_GDP), 3)

1×25×1×36 Array{Float64,4}:
[:, :, 1, 1] =
 63833.7  31849.2  38829.4  95835.1  …  37373.7  2.04753e5  1.13268e6

[:, :, 1, 2] =
 66276.9  31205.6  41874.3  104485.0  …  38404.2  2.13368e5  1.20128e6

[:, :, 1, 3] =
 63192.2  32279.6  41239.0  1.11371e5  …  40112.9  2.07412e5  1.18411e6

...

[:, :, 1, 34] =
 1.68948e5  74889.0  92913.6  2.78457e5  …  83401.4  511694.0  3.4726e6

[:, :, 1, 35] =
 1.85617e5  81144.1  93264.7  3.06283e5  …  89070.4  5.56706e5  3.65423e6

[:, :, 1, 36] =
 2.05683e5  89386.2  1.04464e5  3.69932e5  …  95611.3  6.24112e5  3.92776e6

In [4]:
rows = [("1970s", 1:8), ("1980s", 9:18), ("1990s", 19:28), ("2000s", 28:36)]
columns = ["actual", "kappa1972", "nosectoral", "nosectoral_kappa1972"]

4-element Array{String,1}:
 "actual"              
 "kappa1972"           
 "nosectoral"          
 "nosectoral_kappa1972"

In [7]:
function get_volatility(rows, column)
    results = load("../experiments/baseline/$(column)/results.jld2")["results"]
    real_GDP_model = sum(ImpvolOutput.make_series(results, :real_GDP), 3)
    return [mean(ImpvolOutput.calculate_volatilities(real_GDP_model, parameters, true, row[2])) for row in rows]
end

get_volatility (generic function with 1 method)

In [8]:
using DataFrames
volatility = DataFrame()
volatility[:decades] = [row[1] for row in rows]
for column in columns
    volatility[Symbol(column)] = get_volatility(rows, column)
end


In [9]:
volatility

,decades,actual,kappa1972,nosectoral,nosectoral_kappa1972
1,1970s,0.000847905,0.000878323,0.00111611,0.00125438
2,1980s,0.00102683,0.0011707,0.00118557,0.00160655
3,1990s,0.00106489,0.00161412,0.00180077,0.00286815
4,2000s,0.000739719,0.00223099,0.00176015,0.00321032


In [10]:
stats = volatility
stats[:trade_barriers] = 100 * (stats[:actual] - stats[:kappa1972]) ./ stats[:kappa1972]
stats[:diversification] = 100 * (stats[:nosectoral] - stats[:nosectoral_kappa1972]) ./ stats[:kappa1972]
stats[:specialization] = 100 * (stats[:actual] - stats[:kappa1972] - stats[:nosectoral] + stats[:nosectoral_kappa1972]) ./ stats[:kappa1972]


4-element Array{Float64,1}:
 12.2795 
 23.6716 
 32.101  
 -1.84235

In [11]:
for col in columns
    delete!(volatility, Symbol(col))
end

In [13]:
using CSV
CSV.write("../volatility_by_decade.csv", volatility)

CSV.Sink{Void,DataType}(    CSV.Options:
        delim: ','
        quotechar: '"'
        escapechar: '\\'
        missingstring: ""
        dateformat: nothing
        decimal: '.'
        truestring: 'true'
        falsestring: 'false'
        internstrings: true, IOBuffer(data=UInt8[...], readable=true, writable=true, seekable=true, append=false, size=0, maxsize=Inf, ptr=1, mark=-1), "../volatility_by_decade.csv", 54, true, String["decades", "trade_barriers", "diversification", "specialization"], 4, false, Val{false})